In [2]:
from flask import Flask, render_template, Response, request
import cv2

app = Flask(__name__)

# Haar Cascade를 사용한 얼굴 인식 로드
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
camera = cv2.VideoCapture(0)

@app.route('/')
def index():
    print("index() 실행")
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    print("upload() 실행")
    if 'file' not in request.files:
        return 'No file part'
    file = request.files['file']
    file.save('received_image.jpg')
    return 'File received'

def generate_frames():
    print("generate_frames 함수가 실행되었습니다.")  # 함수 시작 로그

    while True:
        success, frame = camera.read()
        if not success:
            break

        # 얼굴 인식
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)

        # 인식된 얼굴에 사각형을 그리기
        for (x, y, w, h) in faces:
            # (x, y) 좌표에서 시작하여 (x+w, y+h)까지 사각형 그리기
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            
            # 얼굴을 인식했을 때 로그 출력
            print(f"얼굴 인식됨: 위치({x}, {y}), 크기({w}, {h})")

        # JPEG 포맷으로 인코딩
        _, jpeg = cv2.imencode('.jpg', frame)
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + jpeg.tobytes() + b'\r\n')

@app.route('/video_feed')
def video_feed():
    print("video_feed() 실행")
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.124:5000
Press CTRL+C to quit
[2024-09-01 22:52:24,787] ERROR in app: Exception on /upload [POST]
Traceback (most recent call last):
  File "c:\Users\bm838\anaconda3\Lib\site-packages\flask\app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\bm838\anaconda3\Lib\site-packages\flask\app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\bm838\anaconda3\Lib\site-packages\flask\app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\bm838\anaconda3\Lib\site-packages\flask\app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^